# 입문용 이미지 캡셔닝(Image Captioning) 실습 노트북


- 데이터셋: GitHub에서 제공하는 **Flickr8k** 데이터셋 (일부 샘플만 사용)
- 인코더(Encoder): 사전 학습된 **ResNet-18(CNN)**
- 디코더(Decoder): **LSTM 기반 문장 생성기**




In [1]:
# ===== 1. 기본 라이브러리 임포트 =====
import os  # 운영체제 기능(폴더 생성, 경로 처리 등)을 사용하기 위한 모듈
import re  # 정규표현식(텍스트 전처리)에 사용되는 모듈
import zipfile  # zip 파일(압축 파일)을 풀기 위해 사용하는 모듈
import random  # 무작위 샘플 추출, 시드 고정 등에 사용하는 모듈
from collections import Counter  # 단어 빈도수를 세기 위해 사용하는 자료구조

import urllib.request  # 인터넷에서 파일을 다운로드하기 위한 표준 라이브러리 모듈

import numpy as np  # 숫자 계산과 배열 연산을 편리하게 해주는 라이브러리
from PIL import Image  # 이미지 파일을 열고 다루기 위한 라이브러리(Pillow)
import matplotlib.pyplot as plt  # 그래프나 이미지를 화면에 출력하기 위한 라이브러리

import torch  # PyTorch 딥러닝 프레임워크의 핵심 패키지
from torch import nn  # 신경망 레이어를 만들기 위한 모듈
from torch.utils.data import Dataset, DataLoader  # 데이터셋과 배치 생성을 도와주는 클래스들

from torchvision import transforms  # 이미지 전처리(리사이즈, 텐서 변환 등)를 위한 모듈
from torchvision.models import resnet18, ResNet18_Weights  # 사전 학습된 ResNet-18 모델과 그 가중치 설정

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU가 있으면 GPU, 없으면 CPU를 사용하도록 설정
print("사용 중인 디바이스:", device)  # 현재 사용 중인 디바이스를 출력하여 확인


사용 중인 디바이스: cuda


In [2]:
# ===== 2. 재현성을 위한 시드(seed) 고정 =====
def set_seed(seed: int = 42):  # seed 값을 받아서 여러 라이브러리의 난수 발생기를 고정하는 함수 정의
    random.seed(seed)  # 파이썬 기본 random 모듈의 시드를 고정
    np.random.seed(seed)  # 넘파이의 난수 시드를 고정
    torch.manual_seed(seed)  # PyTorch CPU 난수 시드를 고정
    if torch.cuda.is_available():  # 만약 GPU(CUDA)가 사용 가능하다면
        torch.cuda.manual_seed_all(seed)  # 모든 GPU의 난수 시드를 고정

set_seed(42)  # 위에서 정의한 함수를 호출하여 시드를 42로 고정


## 3. GitHub에서 Flickr8k 데이터 다운로드

Flickr8k 데이터셋은 **이미지 캡션 연구**에 많이 사용되는 작은 이미지-문장 쌍 데이터셋입니다.

여기서는 GitHub 저장소인
[`Avaneesh40585/Flickr8k-Dataset`](https://github.com/Avaneesh40585/Flickr8k-Dataset) 의
릴리즈에 올라온 **압축 파일(zip)** 을 직접 다운로드하여 사용합니다.


In [3]:
# ===== 3. Flickr8k 데이터 다운로드 및 압축 해제 =====
import os
import zipfile
import urllib.request

# 전체 데이터셋이 들어갈 기본 폴더
data_dir = "./flickr8k"
os.makedirs(data_dir, exist_ok=True)

# 정상 동작하는 Flickr8k 공식 미러 URL (Jason Brownlee GitHub)
images_zip_url = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip"
text_zip_url   = "https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip"

images_zip_path = os.path.join(data_dir, "Flickr8k_Dataset.zip")
text_zip_path   = os.path.join(data_dir, "Flickr8k_text.zip")

def download_if_not_exists(url, save_path):
    """파일이 없을 때만 다운로드"""
    if not os.path.exists(save_path):
        print(f"다운로드 중: {url}")
        urllib.request.urlretrieve(url, save_path)
        print(f"완료: {save_path}")
    else:
        print(f"이미 존재: {save_path}")

def unzip_if_needed(zip_path, extract_to):
    """압축이 아직 안 풀려 있으면 압축 해제"""
    if not os.path.exists(extract_to):
        print(f"압축 해제 중: {zip_path}")
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(extract_to)
        print(f"압축 해제 완료: {extract_to}")
    else:
        print(f"이미 압축 해제됨: {extract_to}")

# 1) zip 파일 다운로드
download_if_not_exists(images_zip_url, images_zip_path)
download_if_not_exists(text_zip_url,   text_zip_path)

import zipfile
import os

zip_path = "/content/flickr8k/Flickr8k_Dataset.zip"
extract_path = "/content/flickr8k/"

print("압축 해제 중...")
with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(extract_path)

print("완료!")
print("압축 해제 후 폴더 목록:", os.listdir(extract_path))




다운로드 중: https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip
완료: ./flickr8k/Flickr8k_Dataset.zip
다운로드 중: https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip
완료: ./flickr8k/Flickr8k_text.zip
압축 해제 중...
완료!
압축 해제 후 폴더 목록: ['Flickr8k_Dataset.zip', 'Flicker8k_Dataset', 'Flickr8k_text.zip', '__MACOSX']


## 4. 캡션 파일 로드 및 구조 이해

`Flickr8k.token.txt` 파일에는 **이미지 파일 이름과 그 이미지에 대한 여러 문장(캡션)** 이 함께 들어 있습니다.

예시 형식:

```text
1000268201_693b08cb0e.jpg#0\tA child in a pink dress is climbing up a set of stairs in an entry way .
```

- `1000268201_693b08cb0e.jpg` : 이미지 파일 이름
- `#0` : 이 이미지에 대한 0번째 캡션 (한 이미지당 5개의 캡션)
- 그 뒤 : 실제 문장


In [4]:
import zipfile

# 데이터셋 압축 파일 경로 (이미지에서 보인 경로를 기반으로 예시)
# 'data_dir'이 'flickr8k'의 상위 폴더를 가리킨다고 가정합니다.
zip_path = os.path.join(data_dir, "Flickr8k_text.zip")

# 압축을 풀 디렉토리
extract_to_dir = data_dir

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)
    print(f"'{zip_path}' 파일이 '{extract_to_dir}'에 성공적으로 압축 해제되었습니다.")

'./flickr8k/Flickr8k_text.zip' 파일이 './flickr8k'에 성공적으로 압축 해제되었습니다.


In [5]:
import os

# 예시: data_dir이 'flickr8k'의 상위 디렉토리라고 가정
# 사용자 환경에 맞춰 'data_dir' 변수 설정이 필요합니다.
# 만약 'data_dir'이 이미 'flickr8k' 폴더를 가리킨다면, 아래 코드는 필요 없습니다.
# 여기서는 'flickr8k' 폴더 안에 데이터가 있다고 가정하고 경로를 설정합니다.
# **!!! 중요: 실제 환경에 맞게 data_dir 값을 설정해야 합니다 !!!**
# 예를 들어 data_dir이 '/content/' 이고 'flickr8k' 폴더가 그 안에 있다면:
# data_dir = '/content/flickr8k'
# 이 예시에서는 기존 코드의 'data_dir'이 'flickr8k' 폴더의 경로라고 가정하고 진행합니다.

# --- 4. 캡션 파일 로드 ---
# 'Flickr8k_text.zip' 압축을 푼 후, 'Flickr8k.token.txt' 파일의 실제 경로를 확인하여 수정해야 합니다.
# 일반적으로 'data_dir' 안에 압축을 풀면 'Flickr8k.token.txt' 파일이 바로 생깁니다.
# 만약 에러가 발생했다면, 파일 이름이 잘못되었거나 경로가 잘못되었을 가능성이 큽니다.

# 1. 파일 이름이 잘못되었을 경우 (혹시나 하여 오타 수정 가능성 포함):
# captions_file = os.path.join(data_dir, "Flickr8k.token.txt") # 기존 코드

# 2. 파일이 'flickr8k' 폴더 안에 있고, data_dir이 'flickr8k'의 상위 폴더인 경우:
# data_dir이 'flickr8k' 폴더를 포함하는 상위 경로일 경우 아래처럼 수정해야 합니다.
# captions_file = os.path.join(data_dir, "flickr8k", "Flickr8k.token.txt")
# (이 경우 'data_dir'의 정확한 정의가 필요합니다.)

# ***가장 일반적인 해결책: 파일 이름이 정확하다면, 압축을 풀지 않았거나 파일의 실제 위치가 다른 것입니다.***

# **Flickr8k.token.txt 파일을 찾을 수 있는 올바른 경로로 수정하세요.**
# (예시: data_dir이 데이터를 담고 있는 최상위 폴더이고, 그 안에 'Flickr8k.token.txt'가 있다고 가정합니다.)
captions_file = os.path.join(data_dir, "Flickr8k.token.txt") # 압축을 푼 파일 경로를 지정

print("캡션 파일 경로:", captions_file)

try:
    with open(captions_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    print("전체 캡션 라인 수:", len(lines))
    print("앞에서 3줄만 미리 보기:")
    for i in range(3):
        print(lines[i].strip())

except Exception as e:
    print(f"\n파일을 읽는 중 예기치 않은 에러 발생: {e}")

캡션 파일 경로: ./flickr8k/Flickr8k.token.txt
전체 캡션 라인 수: 40460
앞에서 3줄만 미리 보기:
1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .


## 5. 텍스트 전처리 및 이미지-캡션 매핑 만들기

이미지 파일 이름별로 여러 개의 캡션 문장을 모아 두기 위해, 다음과 같은 과정을 거칩니다.

1. 한 줄씩 읽어 **이미지 이름**과 **문장** 부분을 분리합니다.
2. 문장 안의 불필요한 기호(쉼표, 마침표 등)를 제거하고, 모두 소문자로 바꿉니다.
3. 이미지 이름을 key로 하고, 그 이미지에 대한 여러 캡션 리스트를 value로 갖는 딕셔너리를 만듭니다.


In [6]:
# ===== 5. 텍스트 전처리 및 이미지-캡션 딕셔너리 생성 =====
def clean_sentence(sentence: str) -> str:  # 한 문장을 깨끗하게 전처리하는 함수 정의
    sentence = sentence.lower()  # 모든 문자를 소문자로 변환 (예: 'A Dog' -> 'a dog')
    sentence = re.sub(r"[^a-z ]", "", sentence)  # 알파벳 소문자와 공백을 제외한 문자(숫자, 기호 등)를 제거
    sentence = re.sub(r"\s+", " ", sentence).strip()  # 여러 개의 공백을 하나로 줄이고, 양끝 공백 제거
    return sentence  # 전처리가 끝난 문장을 반환

captions_dict = {}  # 이미지 파일 이름을 key, 해당 이미지의 문장 리스트를 value로 저장할 딕셔너리

for line in lines:  # 캡션 파일에서 읽어온 모든 줄을 하나씩 순회
    line = line.strip()  # 줄 끝의 줄바꿈 문자 등을 제거하여 깔끔한 문자열로 만듦
    if len(line) == 0:  # 빈 줄인 경우는 건너뛰기
        continue  # 다음 줄로 넘어감
    image_and_caption = line.split("\t")  # 탭 문자 기준으로 이미지 정보와 문장을 분리
    if len(image_and_caption) != 2:  # 만약 탭으로 나눈 결과가 2개가 아니라면 형식이 이상한 것이므로
        continue  # 해당 줄은 건너뛰고 다음 줄로 이동
    image_id_raw, caption_raw = image_and_caption  # 왼쪽은 이미지+번호, 오른쪽은 문장 부분으로 변수에 저장
    image_filename = image_id_raw.split("#")[0]  # '파일이름#번호' 형태에서 앞부분(파일 이름)만 사용
    cleaned = clean_sentence(caption_raw)  # 위에서 정의한 함수로 문장을 전처리
    if len(cleaned.split()) < 3:  # 단어 수가 너무 적은 문장은 학습에 별 도움이 안 되므로 제외
        continue  # 다음 줄로 넘어감
    captions_dict.setdefault(image_filename, []).append(cleaned)  # 해당 이미지 파일 이름에 문장 추가

print("이미지 개수(캡션 포함):", len(captions_dict))  # 캡션이 있는 이미지가 몇 개인지 출력

# 한 이미지에 어떤 캡션들이 들어 있는지 예시로 하나만 출력
sample_key = next(iter(captions_dict.keys()))  # 딕셔너리에서 임의의 첫 번째 key를 가져옴
print("예시 이미지 파일 이름:", sample_key)  # 선택된 이미지 파일 이름 출력
print("해당 이미지의 캡션들:")  # 그 이미지에 대응되는 문장들을 출력하겠다는 안내 메시지
for c in captions_dict[sample_key]:  # 선택된 이미지에 대한 캡션 리스트를 순회
    print("-", c)  # 각 캡션을 한 줄에 하나씩 출력


이미지 개수(캡션 포함): 8092
예시 이미지 파일 이름: 1000268201_693b08cb0e.jpg
해당 이미지의 캡션들:
- a child in a pink dress is climbing up a set of stairs in an entry way
- a girl going into a wooden building
- a little girl climbing into a wooden playhouse
- a little girl climbing the stairs to her playhouse
- a little girl in a pink dress going into a wooden cabin


## 6. 입문용: 작은 서브셋(subset)만 사용하기

실제 Flickr8k 데이터셋은 8,000장 이상의 이미지를 포함하지만, **수업 실습용** 으로는 너무 무겁습니다.

그래서 여기서는 **임의로 200장의 이미지**만 선택해서, 간단한 모델을 빠르게 학습해 보겠습니다.


In [7]:
# ===== 6. 작은 서브셋 선택 =====
all_image_filenames = list(captions_dict.keys())  # 캡션이 있는 모든 이미지 파일 이름을 리스트로 변환
print("캡션이 있는 전체 이미지 수:", len(all_image_filenames))  # 전체 이미지 개수를 출력

subset_size = 200  # 서브셋으로 사용할 이미지 개수를 200장으로 설정
if len(all_image_filenames) < subset_size:  # 만약 전체 개수가 200보다 작다면
    subset_size = len(all_image_filenames)  # 사용할 개수를 전체 개수로 조정

small_image_filenames = random.sample(all_image_filenames, subset_size)  # 전체 이미지 중에서 무작위로 subset_size개를 뽑음
print("실습에 사용할 이미지 수:", len(small_image_filenames))  # 실제 사용할 이미지 개수 출력


캡션이 있는 전체 이미지 수: 8092
실습에 사용할 이미지 수: 200


## 7. 단어 사전(vocabulary) 만들기

이미지 캡셔닝에서 문장을 다루려면, **단어를 숫자(index)** 로 바꾸는 과정이 필요합니다.

- 특별 토큰(special token)
  - `<pad>` : 빈 자리를 채울 때 사용하는 토큰 (길이를 맞추기 위함)
  - `<start>` : 문장이 시작됨을 알리는 토큰
  - `<end>` : 문장이 끝났음을 알리는 토큰
  - `<unk>` : 사전에 없는 단어를 대신하는 토큰

단어 빈도가 너무 낮은 단어는 모두 `<unk>`로 처리해, 사전의 크기를 적당히 줄입니다.


In [8]:
# ===== 7. 단어 사전 구성 =====
special_tokens = ["<pad>", "<start>", "<end>", "<unk>"]  # 특별한 의미를 가지는 4개의 특수 토큰 리스트

word_counter = Counter()  # 각 단어가 몇 번 등장했는지 세기 위한 Counter 객체
for img in small_image_filenames:  # 선택된 서브셋 이미지들에 대해서만 반복
    for cap in captions_dict[img]:  # 각 이미지에 대해 여러 캡션들을 순회
        for w in cap.split():  # 문장을 공백 기준으로 나누어 단어 리스트를 얻음
            word_counter[w] += 1  # 해당 단어의 등장 빈도를 1 증가시킴

min_freq = 3  # 단어가 최소 몇 번 이상 나타나야 사전에 포함할지 기준 (여기서는 3번 이상)
vocab_words = [w for w, c in word_counter.items() if c >= min_freq]  # 등장 빈도가 기준 이상인 단어만 추려서 리스트 생성
print("기준 이상으로 등장한 단어 수:", len(vocab_words))  # 사전에 포함될 일반 단어 수를 출력

idx2word = []  # 인덱스에서 단어로 바꾸기 위한 리스트(인덱스 -> 단어)
idx2word.extend(special_tokens)  # 앞쪽에 특수 토큰들을 순서대로 추가
idx2word.extend(sorted(vocab_words))  # 나머지 단어들을 정렬하여 뒤에 붙임

word2idx = {w: i for i, w in enumerate(idx2word)}  # 단어에서 인덱스로 바꾸기 위한 딕셔너리(단어 -> 인덱스)

pad_idx = word2idx["<pad>"]  # 패딩 토큰의 인덱스를 변수로 저장 (나중에 손실 계산에서 무시할 때 사용)
start_idx = word2idx["<start>"]  # 문장 시작 토큰의 인덱스
end_idx = word2idx["<end>"]  # 문장 끝 토큰의 인덱스
unk_idx = word2idx["<unk>"]  # 사전에 없는 단어를 대신할 토큰의 인덱스

vocab_size = len(idx2word)  # 최종 단어 사전의 크기(특수 토큰 포함)
print("최종 단어 사전 크기:", vocab_size)  # 사전 크기를 출력


기준 이상으로 등장한 단어 수: 466
최종 단어 사전 크기: 470


In [9]:
# ===== 8. 문장을 숫자 시퀀스로 변환하는 함수 =====
def sentence_to_indices(sentence: str, max_len: int = 20):  # 문장과 최대 길이를 받아서 인덱스 리스트로 변환하는 함수
    tokens = sentence.split()  # 공백 기준으로 단어들을 분리하여 리스트로 만듦
    indices = [start_idx]  # 문장 시작을 의미하는 토큰 인덱스를 맨 앞에 추가
    for w in tokens:  # 문장의 각 단어에 대해 반복
        idx = word2idx.get(w, unk_idx)  # 단어가 사전에 있으면 그 인덱스를, 없으면 <unk> 인덱스를 가져옴
        indices.append(idx)  # 인덱스 리스트에 추가
        if len(indices) >= max_len - 1:  # 이미 충분히 길어졌다면 (마지막에 <end>를 하나 더 붙일 예정)
            break  # 더 이상 단어를 추가하지 않고 반복 종료
    indices.append(end_idx)  # 문장 끝을 의미하는 <end> 토큰 인덱스를 마지막에 추가
    # 길이가 너무 짧으면 뒤쪽을 <pad> 인덱스로 채워서 길이를 맞춤
    if len(indices) < max_len:  # 현재 길이가 최대 길이보다 짧다면
        indices.extend([pad_idx] * (max_len - len(indices)))  # 남은 부분을 모두 <pad>로 채움
    return indices  # 완성된 인덱스 리스트를 반환

# 예시로 한 문장을 숫자 시퀀스로 변환해 보기
example_sentence = captions_dict[small_image_filenames[0]][0]  # 서브셋의 첫 번째 이미지에 대한 첫 번째 캡션 문장을 가져옴
print("예시 원본 문장:", example_sentence)  # 원본 문장을 출력
example_indices = sentence_to_indices(example_sentence, max_len=10)  # 최대 길이를 10으로 제한하여 인덱스 시퀀스로 변환
print("숫자 시퀀스:", example_indices)  # 변환된 인덱스 리스트를 출력
print("다시 단어로:", [idx2word[i] for i in example_indices])  # 인덱스를 다시 단어로 바꿔서 사람이 읽을 수 있게 출력


예시 원본 문장: there are two dogs in the snow and one has something in his mouth
숫자 시퀀스: [1, 411, 18, 436, 121, 198, 408, 369, 15, 2]
다시 단어로: ['<start>', 'there', 'are', 'two', 'dogs', 'in', 'the', 'snow', 'and', '<end>']


## 8. PyTorch Dataset 만들기 (이미지 + 캡션)

딥러닝 학습을 위해서는 데이터를 **(입력, 정답)** 형태로 계속 공급해 주어야 합니다.

- 입력(Input): 전처리된 이미지 텐서
- 정답(Target): 같은 이미지에 대한 캡션 문장(숫자 시퀀스)

PyTorch의 `Dataset` 클래스를 상속하여, 우리가 원하는 형식으로 데이터를 꺼낼 수 있도록 만들어 봅니다.


In [10]:
from torch.utils.data import Dataset
import os

class Flickr8kDataset(Dataset):
    def __init__(self, image_folder, captions_dict, transform=None, max_len=20):
        self.image_folder = image_folder         # 예: "./flickr8k/Flicker8k_Dataset"
        self.captions_dict = captions_dict       # {"파일명.jpg": [token_id,...], ...}
        self.transform = transform
        self.max_len = max_len

        # 1) 캡션에 등장하는 전체 이미지 파일명
        all_image_ids = list(captions_dict.keys())

        # 2) 실제 폴더에 존재하는 파일만 남기기
        valid_image_ids = []
        missing_image_ids = []

        for img_id in all_image_ids:
            img_path = os.path.join(self.image_folder, img_id)
            if os.path.exists(img_path):
                valid_image_ids.append(img_id)
            else:
                missing_image_ids.append(img_id)

        self.image_ids = valid_image_ids


    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.image_folder, img_id)

        # 혹시 모를 예외 상황 방지용 (거의 안 나오겠지만 안전장치)
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Dataset 내부 오류: {img_path} 가 존재하지 않습니다.")

        # 이미지 로드
        from PIL import Image
        image = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        # 캡션 텐서 가져오기 (이미 앞에서 토크나이즈 + 패딩했다고 가정)
        caption = self.captions_dict[img_id]

        return image, caption


## 9. CNN 인코더(Encoder) 정의

이미지 캡셔닝에서 **인코더(Encoder)** 는 이미지를 입력받아, 그 이미지의 특징을 요약한 **벡터(feature vector)** 를 만들어 줍니다.

여기서는 **사전 학습된 ResNet-18** 모델을 사용하여, 마지막 분류 레이어 부분만 제거하고 **512차원 특징 벡터**를 사용합니다.


In [11]:
# ===== 11. CNN 인코더 정의 (ResNet-18) =====
class EncoderCNN(nn.Module):  # PyTorch의 nn.Module을 상속하여 이미지 인코더 클래스를 정의
    def __init__(self, embed_size: int = 256):  # 임베딩 차원(embed_size)을 인자로 받아 초기화
        super().__init__()  # 부모 클래스(nn.Module)의 초기화 메서드 호출
        weights = ResNet18_Weights.DEFAULT  # torchvision에서 제공하는 ResNet-18의 기본 사전 학습 가중치 설정
        resnet = resnet18(weights=weights)  # 사전 학습된 가중치를 가진 ResNet-18 모델 불러오기
        modules = list(resnet.children())[:-1]  # 마지막 분류용 FC 레이어를 제외한 나머지 레이어들만 리스트로 추출
        self.cnn = nn.Sequential(*modules)  # 추출한 레이어들을 nn.Sequential로 묶어서 하나의 모듈로 구성
        self.fc = nn.Linear(resnet.fc.in_features, embed_size)  # ResNet 마지막 특성 차원에서 embed_size로 줄이는 선형 레이어
        self.bn = nn.BatchNorm1d(embed_size)  # 학습 안정화를 위해 배치 정규화 레이어 추가

        for param in self.cnn.parameters():  # 사전 학습된 CNN 가중치들에 대해 반복
            param.requires_grad = False  # 입문용 예제에서는 CNN 부분은 학습하지 않고 고정(freeze)하여 빠르게 학습

    def forward(self, images):  # 순전파(forward) 메서드 정의, 입력은 이미지 텐서
        features = self.cnn(images)  # CNN을 통과시켜 (배치, 채널, 1, 1) 형태의 특징 맵을 얻음
        features = features.view(features.size(0), -1)  # (배치, 채널, 1, 1)을 (배치, 채널) 형태로 펼침
        features = self.fc(features)  # 선형 레이어를 통과시켜 embed_size 차원의 벡터로 변환
        features = self.bn(features)  # 배치 정규화로 분포를 안정화
        return features  # 최종 이미지 특징 벡터를 반환


In [12]:
# ===== 10. Image preprocessing (transform) and DataLoader definition =====
from torchvision import transforms
from torch.utils.data import DataLoader

# 1) 이미지 전처리 파이프라인 정의
image_transform = transforms.Compose([     # 여러 전처리(transform)를 순서대로 적용
    transforms.Resize((224, 224)),         # ResNet 입력 크기에 맞게 224x224로 리사이즈
    transforms.ToTensor(),                 # 이미지를 [0,1] 범위의 텐서(채널, 높이, 너비)로 변환
    transforms.Normalize(                  # ImageNet 통계값으로 정규화
        mean=[0.485, 0.456, 0.406],        # 채널별 평균 (ImageNet)
        std=[0.229, 0.224, 0.225],         # 채널별 표준편차 (ImageNet)
    ),
])

max_caption_len = 20  # 캡션의 최대 길이를 20 단어로 제한
images_folder = "/content/flickr8k/Flicker8k_Dataset"
print(os.path.exists(images_folder))  # True 가 나와야 정상



# 2) Flickr8k 커스텀 데이터셋 생성
dataset = Flickr8kDataset(
    image_folder=images_folder,
    captions_dict=captions_dict,
    transform=image_transform,
    max_len=max_caption_len,
)

print("Number of (image, caption) samples:", len(dataset))  # 6000~8000 사이


# 3) DataLoader 정의
batch_size = 16  # 한 번에 모델에 넣을 배치 크기

dataloader = DataLoader(
    dataset,
    batch_size=batch_size,  # ← 세미콜론(;)이 아니라 쉼표(,) 사용
    shuffle=True,
    num_workers=2,          # 데이터 로딩에 사용할 워커 수 (Colab이면 2~4 정도 권장)
)


True
Number of (image, caption) samples: 8091


## 10. LSTM 디코더(Decoder) 정의

디코더는 인코더가 만든 **이미지 특징 벡터**와 이전까지 생성된 단어들을 이용하여,
다음 단어를 하나씩 예측하는 **문장 생성기**입니다.

1. 단어를 **임베딩(Embedding) 레이어**를 통해 숫자 벡터로 바꾼 뒤,
2. **LSTM** 에 순서대로 넣어 주고,
3. LSTM의 출력을 **Linear 레이어**를 통해 각 단어가 나올 확률로 변환합니다.


In [13]:
# ===== 12. LSTM 디코더 정의 =====
class DecoderRNN(nn.Module):  # PyTorch nn.Module을 상속하여 디코더 클래스를 정의
    def __init__(self, embed_size: int, hidden_size: int, vocab_size: int, num_layers: int = 1):  # 초기화 메서드
        super().__init__()  # 부모 클래스 초기화
        self.embed = nn.Embedding(vocab_size, embed_size)  # 단어 인덱스를 embed_size 차원의 벡터로 바꿔주는 임베딩 레이어
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        # LSTM 레이어 정의 (입력: embed_size, 은닉: hidden_size)
        self.fc = nn.Linear(hidden_size, vocab_size)
        # LSTM 출력을 단어 사전 크기만큼의 로짓(logit)으로 변환하는 선형 레이어

    def forward(self, features, captions):  # 순전파 메서드, features: 이미지 벡터, captions: 정답 캡션 시퀀스
        embeddings = self.embed(captions)  # (배치, 시퀀스 길이) 형태의 캡션 인덱스를 임베딩 벡터로 변환
        features = features.unsqueeze(1)
        # (배치, embed_size)를 (배치, 1, embed_size)로 차원 확장하여 LSTM 첫 입력으로 사용
        inputs = torch.cat((features, embeddings[:, :-1, :]), dim=1)
        # 이미지 특징 뒤에 캡션의 마지막 토큰을 제외한 부분을 이어붙여 입력 시퀀스 생성
        outputs, _ = self.lstm(inputs)  # LSTM에 입력 시퀀스를 넣어 전체 시퀀스에 대한 은닉 상태 출력
        outputs = self.fc(outputs)  # 각 시점의 LSTM 출력을 단어 사전 크기의 로짓으로 변환
        return outputs  # (배치, 시퀀스 길이, vocab_size) 형태의 예측 결과 반환

    def sample(self, features, max_len=20):  # 학습된 모델로부터 실제 문장을 생성하기 위한 메서드
        generated_indices = []  # 생성된 단어 인덱스를 순서대로 저장할 리스트
        inputs = features.unsqueeze(1)  # (배치=1, 1, embed_size) 형태로 LSTM 입력 준비
        states = None  # LSTM의 초기 은닉 상태와 셀 상태는 None으로 두면 자동 초기화
        for _ in range(max_len):  # 최대 max_len 길이만큼 단어를 생성
            outputs, states = self.lstm(inputs, states)  # 현재 입력과 상태를 LSTM에 넣어 한 시점의 출력을 얻음
            outputs = self.fc(outputs.squeeze(1))  # LSTM 출력을 선형 레이어에 통과시켜 단어별 로짓으로 변환
            _, predicted = outputs.max(1)  # 가장 확률이 높은 단어 인덱스를 선택
            generated_indices.append(predicted.item())  # 선택된 인덱스를 리스트에 추가
            if predicted.item() == end_idx:  # 만약 <end> 토큰이 나오면 문장 생성을 멈춤
                break  # 반복문 종료
            inputs = self.embed(predicted).unsqueeze(1)  # 예측된 단어를 임베딩하여 다음 시점의 입력으로 사용
        return generated_indices  # 생성된 단어 인덱스 리스트를 반환


In [14]:
# ===== 13. 모델 인스턴스 생성 및 학습 설정 =====
embed_size = 256  # 이미지 특징 벡터와 단어 임베딩 벡터의 차원을 256으로 설정
hidden_size = 512  # LSTM 은닉 상태의 차원을 512로 설정

encoder = EncoderCNN(embed_size=embed_size).to(device)  # EncoderCNN 인스턴스를 만들고, GPU/CPU 디바이스로 이동
decoder = DecoderRNN(embed_size=embed_size, hidden_size=hidden_size, vocab_size=vocab_size).to(device)  # DecoderRNN 인스턴스를 만들고 디바이스로 이동

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)  # 손실 함수로 다중 클래스 분류에 사용하는 CrossEntropyLoss를 사용, 패딩 토큰은 무시
params = list(decoder.parameters()) + [p for p in encoder.fc.parameters()] + [p for p in encoder.bn.parameters()]
# 학습할 파라미터들만 모아서 리스트로 생성
optimizer = torch.optim.Adam(params, lr=1e-3)  # Adam 옵티마이저를 사용하여 파라미터를 업데이트, 학습률은 0.001로 설정


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 241MB/s]


## 11. 간단한 학습 루프 (입문용으로 1~2 epoch만 돌려보기)

실제 연구에서는 수십 epoch 동안 대량의 데이터를 학습시키지만, 여기서는 **개념 이해**와 **코드 흐름 익히기**가 목적이므로
아주 적은 epoch만 학습시킵니다.


In [15]:
from torch.utils.data import Dataset
import os

class Flickr8kDataset(Dataset):
    def __init__(self, image_folder, captions_dict, transform=None, max_len=20):
        self.image_folder = image_folder         # 예: "./flickr8k/Flicker8k_Dataset"
        self.captions_dict = captions_dict       # {"파일명.jpg": [token_id,...], ...}
        self.transform = transform
        self.max_len = max_len

        # 1) 캡션에 등장하는 전체 이미지 파일명
        all_image_ids = list(captions_dict.keys())

        # 2) 실제 폴더에 존재하는 파일만 남기기
        valid_image_ids = []
        missing_image_ids = []

        for img_id in all_image_ids:
            img_path = os.path.join(self.image_folder, img_id)
            if os.path.exists(img_path):
                valid_image_ids.append(img_id)
            else:
                missing_image_ids.append(img_id)

        self.image_ids = valid_image_ids


    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_path = os.path.join(self.image_folder, img_id)

        # 혹시 모를 예외 상황 방지용 (거의 안 나오겠지만 안전장치)
        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Dataset 내부 오류: {img_path} 가 존재하지 않습니다.")

        # 이미지 로드
        from PIL import Image
        image = Image.open(img_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        # 캡션 텐서 가져오기 (이미 앞에서 토크나이즈 + 패딩했다고 가정)
        caption = self.captions_dict[img_id]

        return image, caption


## 12. 학습된 모델로 캡션 생성해 보기

학습이 끝난 후, 몇 개의 이미지를 골라 **모델이 어떤 문장을 만들어 내는지** 직접 확인해 봅니다.


In [16]:
# ===== 4. 캡션 파일 로드 =====
# 위 셀에서 정의한 captions_file 변수를 그대로 사용합니다.

print("캡션 파일 경로:", captions_file)

try:
    # Flickr8k.token.txt 전체 읽기
    with open(captions_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    print("전체 캡션 라인 수:", len(lines))
    print("앞에서 3줄 미리 보기:")
    for i in range(3):
        print(lines[i].strip())

except Exception as e:
    print(f"\n캡션 파일을 읽는 중 예기치 않은 에러 발생: {e}")
    raise


캡션 파일 경로: ./flickr8k/Flickr8k.token.txt
전체 캡션 라인 수: 40460
앞에서 3줄 미리 보기:
1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .


## 13. 정리

1. GitHub에서 Flickr8k 데이터셋을 다운로드하고, 이미지와 캡션을 로드했습니다.
2. 텍스트를 전처리하여 단어 사전(vocabulary)을 만들고, 문장을 숫자 시퀀스로 변환했습니다.
3. 사전 학습된 ResNet-18을 인코더로 사용해 이미지 특징을 추출했습니다.
4. LSTM 기반 디코더를 사용해, 이미지 특징과 캡션을 이용하여 다음 단어를 예측하는 모델을 만들었습니다.
5. 작은 서브셋에 대해 간단한 학습을 수행하고, 실제로 캡션을 생성해 보았습니다.



In [17]:
# eos